<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing

In [ ]:
!pip install textattack

## Imports

In [1]:
import textattack
import os
import transformers
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019
from textattack.metrics.attack_metrics import (
    AttackQueries,
    AttackSuccessRate,
    WordsPerturbed,
)
import json
import os

2024-05-08 12:56:48.092989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 12:56:48.749389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Helper Functions

In [2]:
def load_model_and_tokenizer(hugging_face_path):
    model = transformers.AutoModelForSequenceClassification.from_pretrained(hugging_face_path)
    tokenizer = transformers.AutoTokenizer.from_pretrained(hugging_face_path)
    model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)
    return model_wrapper


def save_to_file(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)


def save_perturbed_metrics(run_results, drive_path=None):
    os.makedirs('./perturbed_text', exist_ok=True)
    perturbed_examples = []
    attack_success_stats = AttackSuccessRate().calculate(run_results)
    words_perturbed_stats = WordsPerturbed().calculate(run_results)
    words_perturbed_stats.pop('num_words_changed_until_success', None)
    attack_query_stats = AttackQueries().calculate(run_results)
    perturbed_dict = {"attack_success_stats": attack_success_stats, "words_perturbed_stats": words_perturbed_stats,
                      "attack_query_stats": attack_query_stats}
    for result in run_results:
        if isinstance(result, textattack.attack_results.SuccessfulAttackResult):
            perturbed_result = {"original_text": result.original_text(), "perturbed_text": result.perturbed_text(),
                                "gfr": result.goal_function_result_str()}
            perturbed_examples.append(perturbed_result)
    perturbed_dict["perturbed_examples"] = perturbed_examples

    try:
        save_to_file(perturbed_dict, f'perturbed_text/perturbed_{i}.json')
        if drive_path is not None:
            path = os.path.join(drive_path, f'perturbed_{i}.json')
            save_to_file(perturbed_dict, path)
    except:
        print(f"Error while saving perturbed examples on drive at iteration {i}")

## Loading dataset and model
Getting train data in order to find attack transformation to add into the training set

In [3]:
bert_imdb = load_model_and_tokenizer("textattack/bert-base-uncased-imdb")
imdb_dataset_train = HuggingFaceDataset("imdb", split="train", shuffle=True)

/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
textattack: Loading datasets dataset imdb, split train.


# Attack

In [ ]:
batch_size = 10
n_iter = 300

for i in range(15, n_iter):
    attack = BAEGarg2019.build(bert_imdb)
    attack_args = textattack.AttackArgs(num_examples=batch_size, num_examples_offset=i * batch_size, parallel=True)
    attacker = Attacker(attack, imdb_dataset_train, attack_args)
    adv_exp_bae_bert_imdb = attacker.attack_dataset()
    save_perturbed_metrics(adv_exp_bae_bert_imdb, '/run/user/1000/gvfs/google-drive:host=studenti.unipd.it,user=damiano.bertoldo/GVfsSharedWithMe/1OKUVPYQzqhGpqt_wXW2NRFoYukW7q2V7/18czngAsdABbSvUzTbIo8SIRi_G-f8Z7G/')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
2024-05-08 12:57:1

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:28<04:19, 28.86s/it]

[[Negative (100%)]] --> [[Positive (56%)]]

I rented I AM [[CURIOUS-YELLOW]] from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to [[see]] this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary [[denizens]] of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was [[considered]] pornographic. Re

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [01:15<05:00, 37.60s/it]

...

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (92%)]]

..Oh wait, I can! This movie is not for the typical film snob, unless you want to brush up on your typical cinematic definitions, like "continuity editing" and "geographic match". [[I]] couldn't tell where I was in this movie. One second they're in the present, next minute their supposedly in the 70's driving a modern SUV and wearing what looked like to me as 80's style clothing. I think. I couldn't pay long enough attention to it since the acting was just [[horrible]]. I think it only got attention because it has a 3d which I did not watch. If you're a b-movie buff, and by b-movie I mean BAD movie, then this film is for you. It's [[home-movie]] and all non-sense style will keep you laughing for as long as you can stay awake. If your tastes are more for Goddard and Antonioni, though, just [[skip]] this one.

..Oh wait, I can! This movie is not

[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:  30%|███       | 3/10 [00:50<01:57, 16.76s/it]

--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (98%)]]

[[You]] have to admire Brad Sykes even if you don't particularly want to, a man who churns out budget horror after budget horror to less than [[enthusiastic]] receptions. But keeps on doing it all the same. [[Even]] the half-hearted praise than surrounds his Camp Blood films is given grudgingly and I'm as [[guilty]] of this as anyone. Brad normally manages to throw something interesting into the mix, a neat idea, a kooky character, whatever, but without the [[funds]] to take it further than base level, he relies on the audience to cut him some slack and appreciate it for what it is and what it could be. Joe Haggerty gives a spirited and very funny performance as Ebenezer Jackson and its a credit to Sykes that he can sense that this oddball turn is going to work within the framework of the film. Coming to a multiplex near you, in a parallel uni

[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:  40%|████      | 4/10 [01:30<02:15, 22.62s/it]

--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

THE ZOMBIE CHRONICLES <br /><br />Aspect ratio: 1.33:1 (Nu-View 3-D)<br /><br />Sound format: Mono<br /><br />Whilst searching for a (literal) ghost town in the middle of nowhere, a young reporter (Emmy Smith) picks up a grizzled hitchhiker (Joseph Haggerty) who tells her two stories involving flesh-eating zombies reputed to haunt the area.<br /><br />An ABSOLUTE waste of time, hobbled from the outset by Haggerty's painfully amateurish performance in a key role. Worse still, the two stories which make up the bulk of the running time are utterly routine, made worse by indifferent performances and lackluster direction by Brad Sykes, previously responsible for the likes of CAMP BLOOD (1999). This isn't a 'fun' movie in the sense that Ed Wood's movies are 'fun' (he, at least, believed in what he was doing and was sincere in his efforts, despite a lack o

### Try with yelp dataset

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("yelp_polarity", None, "test", shuffle=True)

In [ ]:
attack = GeneticAlgorithmAlzantot2018.build(model_wrapper)
#sesso

attack_args = textattack.AttackArgs(num_examples=10, parallel=True)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

# ADVERSARIAL TRAINING & DATA AUGMENTATION #


In [ ]:
BERTModel = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")

In [ ]:
BERTModel.num_parameters()

In [ ]:
BERTModel

# FineTune function #

In [ ]:
def fineTuneModel(model, train_dataset, val_dataset, epochs=20, batch_s=16):
    for param in model.parameters():
        param.require_grad = False

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_s, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_s)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    loss_fn = torch.nn.CrossEntropyLoss()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Training
    for epoch in range(epochs):
        model.train()

        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Loss calculation and backpropagation
            loss = loss_fn(logits, labels)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
